# Complex

In [1]:
import nest_asyncio
nest_asyncio.apply()

from xrpl.clients import JsonRpcClient
from xrpl.wallet import generate_faucet_wallet

import time
import xrpl

from xrpl.models.transactions import Payment, Memo
from xrpl.transaction import safe_sign_and_autofill_transaction
import binascii
from xrpl.transaction import send_reliable_submission

In [15]:
# create a network client
client = JsonRpcClient("https://s.altnet.rippletest.net:51234")
# Set up middle organization's XRPL account
org_account = generate_faucet_wallet(client)
# Set up voter database (can be replaced with a more secure storage solution)
voter_db = {
    'john': {
        'voted': False,
        'otp': '1234',
        'last_vote_time': 0
    },
    'jane': {
        'voted': False,
        'otp': '5678',
        'last_vote_time': 0
    }
}
for i in voter_db:
    user_wallet = generate_faucet_wallet(client)
    voter_db[i]["wallet"] = user_wallet

#create candidates
candidates_votes ={'ALICE': 0, 'BOB': 0, 'CHARLIE': 0, '': 0}

In [17]:
# Set up time window for voting
#vote_start_time = 1679241600 # March 18th, 2022 00:00:00 UTC
#vote_end_time = 1679328000 # March 19th, 2022 00:00:00 UTC

vote_start_time = int(time.time()) # Current time in seconds since the epoch
vote_end_time = vote_start_time + 86400 

In [18]:
# Define function to check if the user is authenticated
def authenticate_user(username, otp):
    if username in voter_db and voter_db[username]['otp'] == otp:
        return True
    else:
        return False

In [19]:
# Define function to check if the user can vote
def can_vote(username):
    if time.time() >= vote_start_time and time.time() <= vote_end_time:
        if not voter_db[username]['voted']:
            if time.time() - voter_db[username]['last_vote_time'] >= 86400: # One day in seconds
                return True
            else:
                print('\nYou have already voted today. Please try again tomorrow.')
                return False
        else:
            print('\nYou have already voted. Thank you!')
            return False
    else:
        print('\nVoting is closed. Please try again next time.')
        return False


In [20]:
def decoding_string(string):
    vote = string.result["Memos"][0]["Memo"]["MemoData"]
    vote = binascii.unhexlify(vote.encode()).decode('utf8')

    candidates_votes[vote] = candidates_votes[vote] + 1


In [21]:

def cast_vote(username):
    if can_vote(username):
        # Get user's vote
        candidate = input(print('\nPlease enter the candidate you are voting for: '))
        candidate = candidate.upper()
        # Check if candidate is valid
        proven = True
        if candidate not in candidates_votes.keys():
            while proven == True:
                print('\nInvalid candidate. Please try again.')
                candidate = input(print('Please enter the candidate you are voting for: '))
                if candidate in candidates_votes.keys():
                    proven = False

        print("\Processing your vote...")
        # Create a Payment transaction object with a memo
        candidate_encode = binascii.hexlify(candidate.encode()).decode()
        payment_tx = Payment(
            account=voter_db[username]["wallet"].classic_address,
            amount="1",  # Amount to send (in drops)
            destination=org_account.classic_address,
            memos= [Memo(memo_data=candidate_encode)]

        )

        # Sign the transaction with the source account's secret
        signed_tx = safe_sign_and_autofill_transaction(payment_tx, voter_db[username]["wallet"], client)
        tx_response = send_reliable_submission(signed_tx, client)
        tx_response
        decoding_string(tx_response)
        
        #Stated the usrr that voted
        voter_db[username]["voted"]=True
        
        print('\nYour vote has been cast. Thank you!')


In [22]:
# Test the voting system
print('Please enter your username: ')
username = input()
print('Please enter your one-time password (OTP): ')
otp = input()
otp = str(otp)

if authenticate_user(username, otp):
    x = cast_vote(username)
else:
    print('\nInvalid username or OTP. Please try again.')

Please enter your username: 
Please enter your one-time password (OTP): 

Please enter the candidate you are voting for: 
\Processing your vote...
